In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Import Basic Libraries**

**more to be imported as needed**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
plt.style.use('seaborn-whitegrid')

import tensorflow

# **Load Data**

In [3]:
train = pd.read_csv('../input/facial-keypoints-detection/training.zip', compression='zip', header=0, sep=',', quotechar='"')
test = pd.read_csv('../input/facial-keypoints-detection/test.zip', compression='zip', header=0, sep=',', quotechar='"')
IdLookupTable = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv',header=0, sep=',', quotechar='"')
SampleSubmission = pd.read_csv('../input/facial-keypoints-detection/SampleSubmission.csv',header=0, sep=',', quotechar='"')

## **Train**

In [4]:
train

In [5]:
len(train['Image'][1])

In [6]:
train.info()

## **Test**

In [7]:
test

In [8]:
len(test['Image'][1])

In [9]:
test.info()

## **IdLookupTable**

In [10]:
IdLookupTable.info()

# **Missing Data**

## **Train**

In [11]:
import missingno as msno
msno.matrix(train.sample(500),color=(1, 0.38, 0.27))

In [12]:
import missingno as msno
msno.heatmap(train,cmap="RdYlGn")

## **Test**

In [13]:
import missingno as msno
msno.matrix(test.sample(500),color=(1, 0.38, 0.27))

In [14]:
import missingno as msno
msno.heatmap(train,cmap="RdYlGn")

**TEST Set doesn't have NANs, but TRAIN Set has many NANs,**
needs to be handled

# **Handle Missing Value**

Missing values are one of the most common problems you can encounter when you try to prepare your data for machine learning. The reason for the missing values might be human errors,interruptions in the data flow, privacy concerns, and so on. Whatever is the reason, missing values affect the performance of the machine learning models.

#### **[missing-value](https://www.kaggle.com/code/pythonkumar/knn-mean-constant-top-6-imputers) Elaboration**

#### **We should NOT Impute here as these are the X & Y coordinates of presence of a facial feature. Imputing Miising values would result in WRONG Identification points on the images, but NAN values might cause our Algorithm to BREAK**

In [15]:
train=train.fillna(method = 'ffill')
train.isnull().sum().sum()

# **Visualize**

In [16]:
viz = np.array([train['Image'][i].split(' ') for i in range(len(train))],dtype='float')
viz

# **Raw & Reshaped Images**

In [17]:
raw=np.array([img.reshape(96,96,1) for img in viz])
raw[:2]

In [18]:
# Gallery using Matplotlib 
fig, ax = plt.subplots(5,5,figsize = (12,12), dpi = 100)
axes = ax.ravel()

for idx,ax  in enumerate(axes):
    ax.imshow(viz[idx].reshape(96,96,1))
fig.show()

In [19]:
keys = train.drop(['Image'], axis=1)
keys

In [20]:
# Gallery using Matplotlib 
fig, ax = plt.subplots(5,5,figsize = (12,12), dpi = 100)
axes = ax.ravel()

for idx,ax  in enumerate(axes):
    ax.imshow(viz[idx].reshape(96,96,1))
    photo_visualize_pnts = keys.iloc[idx].values
    ax.scatter(photo_visualize_pnts[0::2], photo_visualize_pnts[1::2], c='Red', marker='*')
fig.show()

## **Reconstructing Images**

In [21]:
from  tensorflow.keras.preprocessing.image import array_to_img

image=[array_to_img(viz[i,:].reshape(96,96,1)) for i in range(len(viz))]
image[:5]

## **MakeDir & Save Images**

In [22]:
# Make Dir
import os
if not os.path.exists('./Images/'):
    os.makedirs('./Images/')

# Save Images
for idx,x in enumerate(image):
    x.save(f'./image_{idx}.jpg', 'JPEG')

# **Image Manipulation**

## **Inverting-colors**

The Image's pixel values range from 0 – 225, and We can invert each value by subtracting the value from 255. 

**The provided images are already Negated. Further Inverting will Restore original images**

In [23]:
from PIL import Image
fig, ax = plt.subplots(1,5,figsize = (12,12), dpi = 100)
axes = ax.ravel()
for idx,ax  in enumerate(axes):
    mask=np.full((96,96,1),255)
    ax.imshow((mask-raw[idx]))
fig.show()

## **Rotate**


PIL provides in-built image.rotate(angle) function to rotate an image by an angle in Python

In [24]:
from PIL import Image
fig, ax = plt.subplots(1,5,figsize = (12,12), dpi = 100)
axes = ax.ravel()
for idx,ax  in enumerate(axes):
       ax.imshow((image[idx].rotate(45)))
fig.show()

## **Flip**

In [25]:
# FLIP_TOP_BOTTOM 
from PIL import Image
fig, ax = plt.subplots(1,5,figsize = (12,12), dpi = 100)
axes = ax.ravel()
for idx,ax  in enumerate(axes):
       ax.imshow(image[idx].transpose(method=Image.FLIP_TOP_BOTTOM))
fig.show()

In [26]:
# FLIP_LEFT_RIGHT
from PIL import Image
fig, ax = plt.subplots(1,5,figsize = (12,12), dpi = 100)
axes = ax.ravel()
for idx,ax  in enumerate(axes):
       ax.imshow(image[idx].transpose(method=Image.FLIP_LEFT_RIGHT))
fig.show()

## **Blur**

In [27]:
from PIL import Image
from PIL import ImageFilter
fig, ax = plt.subplots(1,5,figsize = (12,12), dpi = 100)
axes = ax.ravel()
for idx,ax  in enumerate(axes):
       ax.imshow(image[idx].filter(ImageFilter.BLUR))
fig.show()

## **Brightness**

In [28]:
from PIL import ImageEnhance
ig, ax = plt.subplots(1,5,figsize = (12,12), dpi = 100)
axes = ax.ravel()
for idx,ax  in enumerate(axes):
    
    # Creating object of Brightness class
    im = ImageEnhance.Brightness(image[idx])
    
    ax.imshow(im.enhance(2.0))
fig.show()

## **Sharpness**

In [29]:
from PIL import ImageEnhance
ig, ax = plt.subplots(1,5,figsize = (12,12), dpi = 100)
axes = ax.ravel()
for idx,ax  in enumerate(axes):
    
    # Creating object of Sharpness class
    im = ImageEnhance.Sharpness(image[idx])
    
    ax.imshow(im.enhance(-2))
fig.show()

## **Color**

In [30]:
from PIL import ImageEnhance
fig, ax = plt.subplots(1,5,figsize = (12,12), dpi = 100)
axes = ax.ravel()
for idx,ax  in enumerate(axes):
    
    # Creating object of Colour class
    im = ImageEnhance.Color(image[idx])
    
    ax.imshow(im.enhance(5))
fig.show()

## **Contrast**

In [31]:
from PIL import ImageEnhance
fig, ax = plt.subplots(1,5,figsize = (12,12), dpi = 100)
axes = ax.ravel()
for idx,ax  in enumerate(axes):
    
    # Creating object of Contrast class
    im = ImageEnhance.Contrast(image[idx])
    
    ax.imshow(im.enhance(5))
fig.show()

# **Data Preparation**

## **Targets**

In [32]:
# target=train.drop(['Image'],axis=1).columns
# target

## **Get Data from Dataframe**

In [33]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# datagen=ImageDataGenerator(
#                             horizontal_flip=True,
#                             vertical_flip=True,
#                             rescale=1./255,
#                             )

# train_images=datagen.flow_from_dataframe(
#                                     dataframe=train, 
#                                     directory='./Images', 
#                                     x_col='Image', 
#                                     y_col=target,  
#                                     target_size=(96, 96), 
#                                     color_mode='grayscale', 
#                                     classes=None, 
#                                     class_mode='raw', 
#                                     validate_filenames=False,
#                                     )

# train_images

## **Keys**

In [34]:
# keys=np.array(keys)
# keys

## **Train Test Split**

It splits the train data into 4 parts, X_train, X_test, y_train, y_test.

* X_train, y_train first used to train the algorithm.
* X_test is used in that trained algorithms to predict outcomes.
* Once we get the outcomes, we compare it with y_test

In [35]:
# from sklearn.model_selection import train_test_split
# img_train, img_test, keys_train, keys_test = train_test_split(raw, keys, test_size=0.25,shuffle=True)

# **Model**

#### **Inspired by VGG16**

![vgg1](https://viso.ai/wp-content/uploads/2021/10/VGG-16-architecture-of-the-model.jpg)

## **Build**

In [36]:
from keras.layers import Conv2D, MaxPool2D, BatchNormalization, Flatten, Dense, Dropout
from keras.models import Sequential
from tensorflow.keras import regularizers

model = Sequential(name='face-recog')

# Convolution Layer 1
model.add(Conv2D(filters=64,kernel_size=3,strides=1,padding='same',activation='relu',input_shape=(96, 96, 1))),
model.add(MaxPool2D(pool_size=(2, 2))),

# Convolution Layer 2
model.add(Conv2D(filters=128,kernel_size=3,strides=1,padding='same',activation='relu',kernel_regularizer='l1_l2')),
model.add(MaxPool2D(pool_size=(2, 2))),

# Convolution Layer 3
model.add(Conv2D(filters=256,kernel_size=3,strides=1,padding='same',activation='relu',kernel_regularizer='l1_l2')),
model.add(MaxPool2D(pool_size=(2, 2))),

# Convolution Layer 4
model.add(Conv2D(filters=512,kernel_size=3,strides=1,padding='same',activation='relu',kernel_regularizer='l1_l2')),
model.add(MaxPool2D(pool_size=(2, 2))),

# flatten output of Convolutions
model.add(Flatten()),

# Hidden Layer 1
model.add(Dense(units=4096,activation='relu',kernel_regularizer='l1_l2')),
model.add(Dropout(0.25)),

# Hidden Layer 2
model.add(Dense(units=1024,activation='relu',kernel_regularizer='l1_l2')),

# Hidden Layer 3
model.add(Dense(units=256,activation='relu',kernel_regularizer='l1_l2')),
model.add(Dropout(0.25)),

# Output Layer
model.add(Dense(units=30,activation='relu'))

model.summary()

## **Visualize**

In [37]:
# !pip install visualkeras
# import visualkeras
# visualkeras.layered_view(model, legend=True)

In [38]:
from keras.utils.vis_utils import plot_model
plot_model(model, 'model.png',show_shapes=True)

## **Compile**

In [39]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import Accuracy

model.compile(optimizer=Adam(learning_rate = 0.003),
                loss='mae',
                metrics=['accuracy'])


## **Callbacks**

In [40]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

es=EarlyStopping(monitor='val_accuracy', patience=10) # This callback will stop the training when there is no improvement in the loss for three consecutive epochs.,
lrr = ReduceLROnPlateau(monitor='val_accuracy',patience=10) 
cb = [es, lrr]

# **Train**

In [41]:
history=model.fit(
#                     train_images,
                  x=raw,
                  y=keys,
                  callbacks=cb,
                  epochs=50,
                  validation_split=0.2,
#                   use_multiprocessing=True,
#                   steps_per_epoch=1,
#                   shuffle=True,
                )

# **Plotting Accuracy**

In [42]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# **Predict**

In [43]:
# Flatten the Test Images
viz_test = np.array([test['Image'][i].split(' ') for i in range(len(test))],dtype='float')
# viz_test

# Reconstruct the Test Images
raw_test=np.array([img.reshape(96,96,1) for img in viz_test])
raw_test[:2]

In [44]:
pred = model.predict(raw_test)
pred

In [55]:
feature_names = list(IdLookupTable['FeatureName'])
image_ids = list(IdLookupTable['ImageId']-1)
row_ids = list(IdLookupTable['RowId'])

feature_list = [feature_names.index(feature) for feature in feature_names]
    
predictions = [pred[x][y] for x,y in zip(image_ids, feature_list)]
predictions[:5]

# **Suggestions:-**
* Kaggle - https://www.kaggle.com/pythonkumar
* GitHub - https://github.com/KumarPython​
* Twitter - https://twitter.com/KumarPython
* LinkedIn - https://www.linkedin.com/in/kumarpython/

# **Submission**

In [54]:
submission=pd.DataFrame({'RowId': row_ids,
                         'Location': predictions,
                        })
submission
submission.to_csv('submission.csv', index=False)